# 01 DATA INGESTION

In [1]:
import os
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
%pwd

'/Users/chrissunderland/Desktop/starbucks_store_predictions/research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/Users/chrissunderland/Desktop/starbucks_store_predictions'

### Create the entity

In [6]:
from dataclasses import dataclass
from pathlib import Path

Consult config.yaml file

In [7]:
@dataclass(frozen=True)
class DataIngestionConfig:  # this matches the fields in the config.yaml file
    
    root_dir: Path
    acs_source_url: str
    local_zip_file: Path
    acs_data_clean: Path
    acs_demo_housing_clean: Path
    acs_econ_clean: Path
    acs_housing_clean: Path
    acs_social_clean: Path
    yelp_api_endpoint: str
    yelp_data: Path
    yelp_csv: Path

### Create configuration manager

In [8]:
from src.StarbucksProject.constants import *
from src.StarbucksProject.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        
        config = self.config.data_ingestion  # access the "data ingestion" portion of config.yaml

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(root_dir=config.root_dir,
                                                    acs_source_url=config.acs_source_url,
                                                    local_zip_file=config.local_zip_file,
                                                    acs_data_clean=config.acs_data_clean,
                                                    acs_demo_housing_clean=config.acs_demo_housing_clean,
                                                    acs_econ_clean=config.acs_econ_clean,
                                                    acs_housing_clean=config.acs_housing_clean,
                                                    acs_social_clean=config.acs_social_clean,
                                                    yelp_api_endpoint=config.yelp_api_endpoint,
                                                    yelp_data=config.yelp_data,
                                                    yelp_csv=config.yelp_csv)

        return data_ingestion_config

### Create the component

Incorporate code, libraries from '01_data_ingestion_acs' & '01_data_ingestion_yelp' notebooks

In [10]:
from src.StarbucksProject import logger
from src.StarbucksProject.utils.common import get_size

In [1]:
import urllib.request as request
import zipfile
from dotenv import load_dotenv
import requests
import time
import json

In [12]:
class DataIngestion:
    
    def __init__(self, config: DataIngestionConfig):
        
        self.config = config  # access relevant section of config.yaml
    
    def download_file(self):
                
        if not os.path.exists(self.config.local_zip_file):
            
            filename, headers = request.urlretrieve(url=self.config.acs_source_url,
                                                    filename=self.config.local_zip_file)
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_zip_file))}")
    
    def extract_zip_file(self):
        
        unzip_path = self.config.root_dir

        with zipfile.ZipFile(self.config.local_zip_file, 'r') as zip_ref:

            zip_ref.extractall(unzip_path)
        

    def clean_acs_csv(self, df):
        
        clean_df = df.copy()
        
        clean_df.rename(columns={"Label (Grouping)": "ZCTA"}, inplace=True)

        clean_df.dropna(axis=1, how='all', inplace=True)

        cols = clean_df.columns

        clean_df.rename(columns= dict(zip(clean_df.columns[1:], [" ".join(col.split("!!")[1:]) for col in list(cols[1:])])), inplace=True)

        clean_df = clean_df.loc[:, ~clean_df.columns.duplicated()]

        clean_df.iloc[:, 1:] = clean_df.iloc[:, 1:].shift(periods=-1)

        clean_df["ZCTA"] = clean_df["ZCTA"].str.strip()

        clean_df = clean_df[clean_df["ZCTA"].str.startswith("Z")]
        
        clean_df["ZCTA"] = clean_df["ZCTA"].apply(lambda x: x.split()[1])

        clean_df.reset_index(drop=True, inplace=True)

        return clean_df
    
    def clean_all_csvs(self):
        
        co_social = pd.read_csv('artifacts/data_ingestion/acs_data/co_social.csv')
        co_social_clean = self.clean_acs_csv(co_social)
        
        co_econ = pd.read_csv('artifacts/data_ingestion/acs_data/co_econ.csv')
        co_econ_clean = self.clean_acs_csv(co_econ)
        
        co_housing = pd.read_csv('artifacts/data_ingestion/acs_data/co_housing.csv')
        co_housing_clean = self.clean_acs_csv(co_housing)
        
        co_demo_housing = pd.read_csv('artifacts/data_ingestion/acs_data/co_demo_housing.csv')
        co_demo_housing.drop('Total housing units', axis=1, inplace=True)
        co_demo_housing_clean = self.clean_acs_csv(co_demo_housing)
        
        create_directories([self.config.acs_data_clean])
        
        co_social_clean.to_csv(self.config.acs_social_clean)
        co_econ_clean.to_csv(self.config.acs_econ_clean)
        co_housing_clean.to_csv(self.config.acs_housing_clean)
        co_demo_housing_clean.to_csv(self.config.acs_demo_housing_clean)
    
    
    def get_yelp_data(self):
        
        load_dotenv()
        
        
        acs_data = pd.read_csv(self.config.acs_demo_housing_clean)  
        acs_data = acs_data.iloc[:, 1:3]
        
        acs_data['Total population'] = acs_data['Total population'].str.replace(',', '')
        acs_data['Total population'] = acs_data['Total population'].astype(int)
                
        acs_data.sort_values(by='Total population', ascending=False, inplace=True)
        acs_data = acs_data.iloc[:250, :]  # try 300 next
        
        zip_codes = list(acs_data['ZCTA'])
        stores_per_zip = []
        reviews_per_zip = []
        weighted_avg_per_zip = []
        
        for zip_code in zip_codes:
    
            off = 0
            term = "starbucks"
            limit = 50
            biz_list = [] # add stores to this list

            while True:

                API_KEY = os.getenv('yelp_api_key')
                ENDPOINT = self.config.yelp_api_endpoint
                HEADERS = {"accept": "application/json", 
                           "Authorization": f"Bearer {API_KEY}"}
                PARAMETERS = {'location': zip_code,
                              'term': term,
                              'offset': off,
                              'limit': limit}


                response = requests.get(url=ENDPOINT,
                                        params= PARAMETERS,
                                        headers= HEADERS)
                logger.info("API request made")

                json_response = response.json()
                biz_array = json_response['businesses']
                biz_names = [i['name'] for i in biz_array]
                biz_ratings = [i['rating'] for i in biz_array]
                biz_review_count = [i['review_count'] for i in biz_array]
                biz_zips = [i['location']['zip_code'] for i in biz_array]
                biz_addresses = [",".join(i['location']['display_address']) for i in biz_array]

                biz_data = list(zip(biz_names, biz_ratings, biz_review_count, biz_zips, biz_addresses))    
                # filter out the Starbucks stores that aren't in the zip code of interest
                filtered_data = [i for i in biz_data if (term in i[0].lower()) and (i[3] == str(zip_code))]

                if len(filtered_data) == 0:
                    logger.info("Broke out of while loop - last request didn't return any stores in current zip code")
                    break  

                for shop in filtered_data:
                    print("store in zip = ", shop)
                    biz_list.append(shop)

                if len([i[3] for i in biz_data[25:] if i[3] == str(zip_code) and term in i[0].lower()]) == 0: 
                    print("Broke out of while loop - last 25 results didn't include a store in current zip code")
                    break

                off += limit  # make another HTTP request for the same zip code

                time.sleep(2)
        
            try: 

                # summarize Yelp review history to assess how Starbucks has performed in the zip code
                df = pd.DataFrame(biz_list, columns=['store_name', 'avg_rating', 'reviews', 'zip', 'address'])
                df['store_total_reviews'] = df.groupby('address')['reviews'].cumsum()
                df['store_weight'] = df['avg_rating'] * df['reviews']
                df['store_total_weight'] =  df.groupby('address')['store_weight'].cumsum()
                df['avg_rating_weighted'] = round(df['store_total_weight'] / df['store_total_reviews'], 2)
                df = df.drop_duplicates(subset=['address'], keep='last')
                df.drop(['avg_rating', 'reviews', 'address', 'store_weight', 'store_total_weight'], axis=1, inplace=True)
                df['zip_average_contribution'] = round(df['store_total_reviews'] * df['avg_rating_weighted'], 2)

                stores_per_zip.append(df.shape[0])
                reviews_per_zip.append(df['store_total_reviews'].sum())
                weighted_avg_per_zip.append(round(df['zip_average_contribution'].sum() / df['store_total_reviews'].sum(), 3))

            except Exception as e:
                logger.exception(f"{e}")
                logger.info("Current zip code didn't contain any Starbucks locations")
                stores_per_zip.append(0)
                reviews_per_zip.append(0)
                weighted_avg_per_zip.append(0)       

            print("\n")
        
        
        starbucks_data = list(zip(zip_codes, stores_per_zip, reviews_per_zip, weighted_avg_per_zip))
        
        starbucks_df = pd.DataFrame(starbucks_data, columns=['zip', 
                                                             'total_stores', 
                                                             'total_reviews', 
                                                             'review_weighted_avg'])
 
        create_directories([self.config.yelp_data])       
        starbucks_df.to_csv(self.config.yelp_csv, index=False)

### Create the pipeline

In [13]:
try:
    
    config = ConfigurationManager()
    
    data_ingestion_config = config.get_data_ingestion_config()
    
    data_ingestion = DataIngestion(config = data_ingestion_config)
    
    data_ingestion.download_file()  # method of component class
    
    data_ingestion.extract_zip_file()
    
    data_ingestion.clean_all_csvs()
    
    data_ingestion.get_yelp_data()
             
except Exception as e:
    
    raise e

[2024-07-09 20:10:48,175: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-07-09 20:10:48,177: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-09 20:10:48,180: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-09 20:10:48,181: INFO: common: created directory at: artifacts]
[2024-07-09 20:10:48,181: INFO: common: created directory at: artifacts/data_ingestion]
[2024-07-09 20:10:48,182: INFO: 3219891120: File already exists of size: ~ 1857 KB]
[2024-07-09 20:10:48,710: INFO: common: created directory at: artifacts/data_ingestion/acs_data_clean]
[2024-07-09 20:10:49,660: INFO: 3219891120: API request made]
store in zip =  ('Starbucks', 2.5, 11, '80134', '12227 Pine Bluffs Way,Ste 101,Parker, CO 80134')
store in zip =  ('Starbucks', 3.0, 2, '80134', '12959 S Parker Rd,King Soopers-Parker #75,Parker, CO 80134')
store in zip =  ('Starbucks', 3.2, 22, '80134', '11150 S Twenty Mile Rd,Target,Parker, CO 80134')
store in zip =  ('Starbuck

[2024-07-09 20:11:03,326: INFO: 3219891120: API request made]
store in zip =  ('Starbucks', 1.9, 16, '80011', '14901 E Colfax Ave,Aurora, CO 80011')
store in zip =  ('Starbucks', 2.0, 60, '80011', '3455 N Salida St,Salida Shops,Aurora, CO 80011')
store in zip =  ('Starbucks', 1.9, 45, '80011', '12700 E Colfax Ave,Aurora, CO 80011')
Broke out of while loop - last 25 results didn't include a store in current zip code


[2024-07-09 20:11:04,247: INFO: 3219891120: API request made]
store in zip =  ('Starbucks', 5.0, 1, '80918', '5240 N Academy Blvd,Colorado Springs, CO 80918')
store in zip =  ('Starbucks', 4.2, 53, '80918', '4825 N Academy Blvd,Colorado Springs, CO 80918')
store in zip =  ('Starbucks', 2.6, 23, '80918', '6815 N Academy Blvd,N/A,Colorado Springs, CO 80918')
store in zip =  ('Starbucks', 3.8, 4, '80918', '3650 Austin Bluffs Pkwy,King Soopers-Colorado Springs #130,Colorado Springs, CO 80918')
store in zip =  ('Starbucks', 2.7, 11, '80918', '4296 Austin Bluffs Pkwy,Colorado Sp

[2024-07-09 20:11:12,656: INFO: 3219891120: Current zip code didn't contain any Starbucks locations]


[2024-07-09 20:11:13,464: INFO: 3219891120: API request made]
store in zip =  ('Starbucks', 3.0, 2, '80401', '1200 16th St,Sodexo @ Colorado School of Mines,Golden, CO 80401')
store in zip =  ('Starbucks', 3.2, 28, '80401', '17224 S Golden Rd,Golden, CO 80401')
store in zip =  ('Starbucks', 3.7, 35, '80401', '1301 Washington Ave,TBD,Golden, CO 80401')
store in zip =  ('Starbucks', 2.3, 3, '80401', '17171 S Golden Rd,King Soopers-Golden, CO #82,Golden, CO 80401')
store in zip =  ('Starbucks', 2.3, 11, '80401', '14500 W Colfax Ave,Unit B1,Target,Lakewood, CO 80401')
Broke out of while loop - last 25 results didn't include a store in current zip code


[2024-07-09 20:11:14,181: INFO: 3219891120: API request made]
store in zip =  ('Starbucks', 4.5, 2, '80550', '1535 Main St,Safeway,Windsor, CO 80550')
store in zip =  ('Starbucks', 2.7, 6, '80550', '1570 Main St,Windsor, CO 80550')
Broke o

[2024-07-09 20:11:23,308: INFO: 3219891120: Current zip code didn't contain any Starbucks locations]


[2024-07-09 20:11:24,116: INFO: 3219891120: API request made]
store in zip =  ('Starbucks', 3.7, 3, '80210', '2055 E Evans Ave,Denver, CO 80210')
store in zip =  ('Starbucks', 3.1, 38, '80210', '2000 S University Blvd,#B,Denver, CO 80210')
store in zip =  ('Starbucks', 2.8, 19, '80210', '2700 S Colorado Blvd,University Hills Mall,Denver, CO 80210')
Broke out of while loop - last 25 results didn't include a store in current zip code


[2024-07-09 20:11:24,830: INFO: 3219891120: API request made]
store in zip =  ('Starbucks', 3.5, 28, '80909', '405 N Union Blvd,Colorado Springs, CO 80909')
Broke out of while loop - last 25 results didn't include a store in current zip code


[2024-07-09 20:11:25,649: INFO: 3219891120: API request made]
store in zip =  ('Starbucks', 3.5, 38, '80128', '7403 W Chatfield Ave,Littleton, CO 80128')
store in zip =  ('Starbucks', 2.3, 3, '80128', '6760 S Pierce

[2024-07-09 20:11:35,138: INFO: 3219891120: API request made]
store in zip =  ('Starbucks', 3.1, 11, '80104', '880 S Perry St,Safeway,Castle Rock, CO 80104')
store in zip =  ('Starbucks', 3.0, 3, '80104', '750 S Ridge Rd,King Soopers - Castle Rock #132,Castle Rock, CO 80104')
Broke out of while loop - last 25 results didn't include a store in current zip code


[2024-07-09 20:11:35,991: INFO: 3219891120: API request made]
[2024-07-09 20:11:35,993: INFO: 3219891120: Broke out of while loop - last request didn't return any stores in current zip code]
[2024-07-09 20:11:35,995: ERROR: 3219891120: cumsum is not supported for object dtype]
Traceback (most recent call last):
  File "/Users/chrissunderland/opt/anaconda3/envs/starbucks_project/lib/python3.10/site-packages/pandas/core/groupby/generic.py", line 529, in _cython_transform
    result = self._grouper._cython_operation(
  File "/Users/chrissunderland/opt/anaconda3/envs/starbucks_project/lib/python3.10/site-packages/pandas/core/groupby

...

[2024-07-09 20:13:47,233: INFO: 3219891120: Current zip code didn't contain any Starbucks locations]


[2024-07-09 20:13:47,958: INFO: 3219891120: API request made]
[2024-07-09 20:13:47,960: INFO: 3219891120: Broke out of while loop - last request didn't return any stores in current zip code]
[2024-07-09 20:13:47,962: ERROR: 3219891120: cumsum is not supported for object dtype]
Traceback (most recent call last):
  File "/Users/chrissunderland/opt/anaconda3/envs/starbucks_project/lib/python3.10/site-packages/pandas/core/groupby/generic.py", line 529, in _cython_transform
    result = self._grouper._cython_operation(
  File "/Users/chrissunderland/opt/anaconda3/envs/starbucks_project/lib/python3.10/site-packages/pandas/core/groupby/ops.py", line 831, in _cython_operation
    return cy_op.cython_operation(
  File "/Users/chrissunderland/opt/anaconda3/envs/starbucks_project/lib/python3.10/site-packages/pandas/core/groupby/ops.py", line 550, in cython_operation
    return self._cython_op_nd

[2024-07-09 20:13:51,674: INFO: 3219891120: API request made]
[2024-07-09 20:13:51,675: INFO: 3219891120: Broke out of while loop - last request didn't return any stores in current zip code]
[2024-07-09 20:13:51,677: ERROR: 3219891120: cumsum is not supported for object dtype]
Traceback (most recent call last):
  File "/Users/chrissunderland/opt/anaconda3/envs/starbucks_project/lib/python3.10/site-packages/pandas/core/groupby/generic.py", line 529, in _cython_transform
    result = self._grouper._cython_operation(
  File "/Users/chrissunderland/opt/anaconda3/envs/starbucks_project/lib/python3.10/site-packages/pandas/core/groupby/ops.py", line 831, in _cython_operation
    return cy_op.cython_operation(
  File "/Users/chrissunderland/opt/anaconda3/envs/starbucks_project/lib/python3.10/site-packages/pandas/core/groupby/ops.py", line 550, in cython_operation
    return self._cython_op_ndim_compat(
  File "/Users/chrissunderland/opt/anaconda3/envs/starbucks_project/lib/python3.10/site-pack

[2024-07-09 20:13:55,060: ERROR: 3219891120: cumsum is not supported for object dtype]
Traceback (most recent call last):
  File "/Users/chrissunderland/opt/anaconda3/envs/starbucks_project/lib/python3.10/site-packages/pandas/core/groupby/generic.py", line 529, in _cython_transform
    result = self._grouper._cython_operation(
  File "/Users/chrissunderland/opt/anaconda3/envs/starbucks_project/lib/python3.10/site-packages/pandas/core/groupby/ops.py", line 831, in _cython_operation
    return cy_op.cython_operation(
  File "/Users/chrissunderland/opt/anaconda3/envs/starbucks_project/lib/python3.10/site-packages/pandas/core/groupby/ops.py", line 550, in cython_operation
    return self._cython_op_ndim_compat(
  File "/Users/chrissunderland/opt/anaconda3/envs/starbucks_project/lib/python3.10/site-packages/pandas/core/groupby/ops.py", line 329, in _cython_op_ndim_compat
    res = self._call_cython_op(
  File "/Users/chrissunderland/opt/anaconda3/envs/starbucks_project/lib/python3.10/site-p

[2024-07-09 20:13:57,196: INFO: 3219891120: Current zip code didn't contain any Starbucks locations]


[2024-07-09 20:13:57,921: INFO: 3219891120: API request made]
[2024-07-09 20:13:57,923: INFO: 3219891120: Broke out of while loop - last request didn't return any stores in current zip code]
[2024-07-09 20:13:57,925: ERROR: 3219891120: cumsum is not supported for object dtype]
Traceback (most recent call last):
  File "/Users/chrissunderland/opt/anaconda3/envs/starbucks_project/lib/python3.10/site-packages/pandas/core/groupby/generic.py", line 529, in _cython_transform
    result = self._grouper._cython_operation(
  File "/Users/chrissunderland/opt/anaconda3/envs/starbucks_project/lib/python3.10/site-packages/pandas/core/groupby/ops.py", line 831, in _cython_operation
    return cy_op.cython_operation(
  File "/Users/chrissunderland/opt/anaconda3/envs/starbucks_project/lib/python3.10/site-packages/pandas/core/groupby/ops.py", line 550, in cython_operation
    return self._cython_op_nd

[2024-07-09 20:13:59,770: INFO: 3219891120: Current zip code didn't contain any Starbucks locations]


[2024-07-09 20:14:00,481: INFO: 3219891120: API request made]
store in zip =  ('Starbucks', 1.8, 6, '80446', '1001 Thompson Rd,City Market-Granby #446,Granby, CO 80446')
Broke out of while loop - last 25 results didn't include a store in current zip code


[2024-07-09 20:14:01,171: INFO: 3219891120: API request made]
[2024-07-09 20:14:01,173: INFO: 3219891120: Broke out of while loop - last request didn't return any stores in current zip code]
[2024-07-09 20:14:01,175: ERROR: 3219891120: cumsum is not supported for object dtype]
Traceback (most recent call last):
  File "/Users/chrissunderland/opt/anaconda3/envs/starbucks_project/lib/python3.10/site-packages/pandas/core/groupby/generic.py", line 529, in _cython_transform
    result = self._grouper._cython_operation(
  File "/Users/chrissunderland/opt/anaconda3/envs/starbucks_project/lib/python3.10/site-packages/pandas/core/groupby/ops

### Modularize the code (once working)